In [1]:
!pip install numpy opencv-python

In [2]:
# import cv2
# import numpy as np
# import time

# print("""

# Harry :  Hey !! Would you like to try my invisibility cloak ??

#          Its awesome !!

        
#          Prepare to get invisible .....................
#     """)


# cap = cv2.VideoCapture(0)
# time.sleep(3)
# background=0
# for i in range(30):
# 	ret,background = cap.read()

# background = np.flip(background,axis=1)

# while(cap.isOpened()):
# 	ret, img = cap.read()
	
# 	# Flipping the image (Can be uncommented if needed)
# 	img = np.flip(img,axis=1)
	
# 	# Converting image to HSV color space.
# 	hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
# 	value = (35, 35)
	
# 	blurred = cv2.GaussianBlur(hsv, value,0)
	
# 	# Defining lower range for red color detection.
# 	lower_red = np.array([0,120,70])
# 	upper_red = np.array([10,255,255])
# 	mask1 = cv2.inRange(hsv,lower_red,upper_red)
	
# 	# Defining upper range for red color detection
# 	lower_red = np.array([170,120,70])
# 	upper_red = np.array([180,255,255])
# 	mask2 = cv2.inRange(hsv,lower_red,upper_red)
	
# 	# Addition of the two masks to generate the final mask.
# 	mask = mask1+mask2
# 	mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((5,5),np.uint8))
	
# 	# Replacing pixels corresponding to cloak with the background pixels.
# 	img[np.where(mask==255)] = background[np.where(mask==255)]
# 	cv2.imshow('Display',img)
# 	k = cv2.waitKey(10)
# 	if k == 27:
# 		break


In [3]:
import cv2
import numpy as np
import time

print("""

Harry :  Hey !! Would you like to try my invisibility cloak ??

         Its awesome !!

         Prepare to get invisible .....................
    """)

# Capture video
cap = cv2.VideoCapture(0)
time.sleep(3)

# Capture static background (important: keep camera still)
for i in range(60):
    ret, background = cap.read()
background = np.flip(background, axis=1)

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break
    
    img = np.flip(img, axis=1)  # Flip for natural viewing
    
    # Convert to HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # ---- STEP 1: Define cloak color range (example: red) ----
    lower_red1 = np.array([0, 120, 70])
    upper_red1 = np.array([10, 255, 255])
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)

    lower_red2 = np.array([170, 120, 70])
    upper_red2 = np.array([180, 255, 255])
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)

    mask = mask1 + mask2

    # ---- STEP 2: Refine mask ----
    kernel = np.ones((3,3), np.uint8)
    
    # Remove small noise
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=2)
    # Fill small holes
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    # Expand mask slightly
    mask = cv2.dilate(mask, kernel, iterations=1)
    # Smooth edges but keep them sharp
    mask = cv2.bilateralFilter(mask, 9, 75, 75)

    # ---- STEP 3: Feather edges ----
    mask_f = cv2.GaussianBlur(mask, (21,21), 11)
    mask_f = mask_f.astype(float) / 255.0
    mask_inv_f = 1 - mask_f

    # ---- STEP 4: Segment regions ----
    res1 = (background.astype(float) * mask_f[...,None])
    res2 = (img.astype(float) * mask_inv_f[...,None])
    final_output = (res1 + res2).astype(np.uint8)

    # Display output
    cv2.imshow("Invisibility Cloak", final_output)

    # Exit on 'Esc'
    k = cv2.waitKey(1)
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()




Harry :  Hey !! Would you like to try my invisibility cloak ??

         Its awesome !!

         Prepare to get invisible .....................
    


KeyboardInterrupt: 